In [ ]:
from keras import Sequential, Model
from keras.applications import Xception
from keras.layers import Flatten, Dense, Softmax, GlobalAvgPool2D
from keras.preprocessing import image
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
model = Sequential()
model.add(Xception(include_top=False, weights=None, input_shape=(299, 299, 3), pooling='avg'))
model.add(Dense(43))
model.add(Softmax())
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

In [ ]:
trainDataFrame = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
testDataFrame = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

trainDataFrame['ClassId'] = trainDataFrame['ClassId'].astype(str)
testDataFrame['ClassId'] = testDataFrame['ClassId'].astype(str)

for i in range(0, len(trainDataFrame['ClassId'])):
    if len(trainDataFrame['ClassId'][i]) == 1:
        trainDataFrame['ClassId'][i] = '0' + trainDataFrame['ClassId'][i]
for i in range(0, len(testDataFrame['ClassId'])):
    if len(testDataFrame['ClassId'][i]) == 1:
        testDataFrame['ClassId'][i] = '0' + testDataFrame['ClassId'][i]

print(trainDataFrame['ClassId'][20], trainDataFrame['Path'][20])
img = Image.open('../input/gtsrb-german-traffic-sign/' + trainDataFrame['Path'][20])
plt.imshow(img)

trainDataGen = image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2
)
testDataGen = image.ImageDataGenerator(
    rescale=1./255
)

trainDataGenerator = trainDataGen.flow_from_dataframe(
    dataframe=trainDataFrame,
    directory='../input/gtsrb-german-traffic-sign/',
    x_col='Path',
    y_col='ClassId',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)
testDataGenerator = testDataGen.flow_from_dataframe(
    dataframe=testDataFrame,
    directory='../input/gtsrb-german-traffic-sign/',
    x_col='Path',
    y_col='ClassId',
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical'
)

In [ ]:
history = model.fit(trainDataGenerator, epochs=2, verbose=1)

In [ ]:
model.evaluate(testDataGenerator, verbose=1)

In [ ]:
test_batch = 2
plt.figure(figsize=(64, 64))
plt.tight_layout()
for i in range(0, 16):
    plt.subplot(16, 2, 2 * i + 1), plt.axis('off')
    plt.imshow(testDataGenerator[test_batch][0][i])
    pre = model.predict_classes(testDataGenerator[test_batch][0])
    img = Image.open('../input/gtsrb-german-traffic-sign/Meta/' + str(pre[i]) + '.png')
    plt.subplot(16, 2, 2 * i + 2), plt.axis('off')
    plt.imshow(img)

In [ ]:
model.save('GTSRB.h5')